In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor 

C:\Users\Vadim\Miniconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Vadim\Miniconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [74]:
df = pd.read_csv("melb_data.csv")
pd.options.display.max_columns = 30
df.head(3)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [69]:
# numerical data descripion
df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [75]:
# Drop some columns and rmpty data 
columns_to_drop = ["Suburb", "Address", "Type", "Method", "SellerG", "Date", "Postcode", "Regionname", "CouncilArea"]
df_cleaned = df.drop(columns=columns_to_drop)
df_cleaned = df_cleaned.dropna()
df_cleaned.shape

(6830, 12)

In [201]:
train_df = df_cleaned.head(5000)
train_Y = train_df["Price"] 
train_X = train_df.drop(columns=["Price"])

test_df = df_cleaned.tail(df_cleaned.shape[0]-5000)
test_Y = test_df["Price"] 
test_X = test_df.drop(columns=["Price"])

## Create first model

In [202]:
model = DecisionTreeRegressor(min_samples_split=10)
model.fit(X=train_X, y=train_Y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [99]:
# Check importance
np.dstack(([model.feature_importances_], [train_X.columns]))

array([[[0.012662536328497589, 'Rooms'],
        [0.0741309748851917, 'Distance'],
        [0.0056602920931579205, 'Bedroom2'],
        [0.02278117403651216, 'Bathroom'],
        [0.00857323762062903, 'Car'],
        [0.06388589011294811, 'Landsize'],
        [0.4693600095315557, 'BuildingArea'],
        [0.13989763381819034, 'YearBuilt'],
        [0.12593231849486017, 'Lattitude'],
        [0.06353124441267342, 'Longtitude'],
        [0.01358468866578386, 'Propertycount']]], dtype=object)

In [203]:
# the coefficient of determination R^2
model.score(X=test_X, y=test_Y)

0.5035793561746548

___

In [204]:
# Drop not important features
not_important_columns = train_X.columns[model.feature_importances_ < 0.05]
train_X = train_X.drop(columns=not_important_columns)
test_X = test_X.drop(columns=not_important_columns)

In [205]:
model.fit(X=train_X, y=train_Y)
# check R^2 again
model.score(X=test_X, y=test_Y) # score downgraded

0.507431129053608

### Model 2

In [24]:
# Columns for model
feature_columns = ["Price", "Rooms", "Landsize", "BuildingArea", "Lattitude", "Longtitude"]
sub_df = df[feature_columns]
# drop rows with empty values
sub_df = sub_df.dropna()

,Price,Rooms,Landsize,BuildingArea,Lattitude,Longtitude
count,7.130000e+03,7130.000000,7130.000000,7130.000000,7130.000000,7130.000000
mean,1.079923e+06,2.977419,490.469425,151.967650,-37.808050,144.991662
std,6.746922e+05,0.970989,1041.681066,541.014538,0.080114,0.105051
min,1.310000e+05,1.000000,0.000000,0.000000,-38.164920,144.542370
25%,6.310000e+05,2.000000,164.000000,93.000000,-37.856797,144.925485
50%,8.950000e+05,3.000000,400.000000,126.000000,-37.802400,144.996800
75%,1.335000e+06,4.000000,638.750000,174.000000,-37.756978,145.056000
max,9.000000e+06,8.000000,44500.000000,44515.000000,-37.408530,145.526350


In [25]:
sub_df.shape

(7130, 6)

In [206]:
# Shuffle data
sub_df.sample(frac=1)
# 5000 samples for train model
train_df = sub_df.head(5000)
test_df = sub_df.tail(sub_df.shape[0]-5000)

In [207]:
# Trainind data
Y = train_df["Price"]
X = train_df.drop(columns=["Price"])

In [208]:
# Test Data
test_Y = test_df["Price"]
test_X = test_df.drop(columns=["Price"])

In [209]:
# Create and train model
dt_model = DecisionTreeRegressor(min_samples_split=15)
dt_model.fit(X=X, y=Y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=15,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [210]:
# Importance of each input feature
dt_model.feature_importances_

array([0.02821952, 0.09781667, 0.52135883, 0.19475788, 0.15784711])

In [211]:
# the coefficient of determination R^2
dt_model.score(X=test_X, y=test_Y) # returns bad score

0.5013595413461207

In [212]:
dt_model.score(X=X, y=Y)

0.9027793217588891

### Try convert data

In [106]:
from sklearn.preprocessing import LabelEncoder


In [107]:
df.head(2)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0


In [213]:
df_cat = df.copy()
df_cat = df_cat.drop(columns=["Address", "SellerG", "Date", "CouncilArea"])
df_cat = df_cat.dropna()
df_cat.head(3)

,Suburb,Rooms,Type,Price,Method,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,2,h,1035000.0,S,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,3,h,1465000.0,SP,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,Northern Metropolitan,4019.0
4,Abbotsford,4,h,1600000.0,VB,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,Northern Metropolitan,4019.0


In [219]:
encoders = {}
encode_columns = ["Suburb", "Type", "Method", "Regionname"]
for column in encode_columns:
    encoder = LabelEncoder()
    encoded_vals = encoder.fit_transform(df_cat[column])
    df_cat[column] = encoded_vals
    encoders.update({column: encoder})

In [220]:
# dataframe with numerical data
df_cat.head()

,Suburb,Rooms,Type,Price,Method,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Regionname,Propertycount
1,0,2,0,1035000.0,1,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,2,4019.0
2,0,3,0,1465000.0,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,2,4019.0
4,0,4,0,1600000.0,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,2,4019.0
6,0,3,0,1876000.0,1,2.5,3067.0,4.0,2.0,0.0,245.0,210.0,1910.0,-37.8024,144.9993,2,4019.0
7,0,2,0,1636000.0,1,2.5,3067.0,2.0,1.0,2.0,256.0,107.0,1890.0,-37.8060,144.9954,2,4019.0


In [221]:
df_cat.shape

(6830, 17)

In [222]:
# Trainind data
train_df = df_cat.head(5000)
test_df = df_cat.tail(df_cat.shape[0]-5000)

Y = train_df["Price"]
X = train_df.drop(columns=["Price"])
# Test Data
test_Y = test_df["Price"]
test_X = test_df.drop(columns=["Price"])

In [233]:
# Create new model
dt_model_v2 = DecisionTreeRegressor(min_samples_split=50)
dt_model_v2.fit(X=X, y=Y)
dt_model_v2.get_depth()

17

In [166]:
# feature importance
np.dstack(([dt_model_v2.feature_importances_], [X.columns]))

array([[[0.011005663439135475, 'Suburb'],
        [0.009974624042921786, 'Rooms'],
        [0.07340151744142497, 'Type'],
        [0.005415036383272461, 'Method'],
        [0.1072805469206137, 'Distance'],
        [0.07556678748717505, 'Postcode'],
        [0.007087260994027301, 'Bedroom2'],
        [0.022737092374549187, 'Bathroom'],
        [0.005225135001076477, 'Car'],
        [0.051730837216435864, 'Landsize'],
        [0.44250652447201494, 'BuildingArea'],
        [0.07550455919725879, 'YearBuilt'],
        [0.07072379739575106, 'Lattitude'],
        [0.027661188053501908, 'Longtitude'],
        [0.001820258316886996, 'Regionname'],
        [0.012359171263954118, 'Propertycount']]], dtype=object)

In [236]:
dt_model_v2.score(test_X, test_Y) # R^2 score improved

0.5810112809582078

In [235]:
dt_model_v2.score(X, Y) # 

0.837280439934783

In [229]:
# dataframe with test values and true values
predicted_price = dt_model_v2.predict(test_X)
pd.DataFrame({"True_val": test_Y, "Predicted": np.array(predicted_price, dtype=int), "Difference": np.array((test_Y - predicted_price), dtype=int)})

,True_val,Predicted,Difference
9672,840000.0,729450,110550
9673,1071888.0,833499,238388
9674,790000.0,550288,239711
9675,820000.0,821162,-1162
9676,1218500.0,1085000,133500
...,...,...,...
13572,650000.0,446929,203070
13573,635000.0,626500,8500
13576,1031000.0,1368333,-337333
13578,2500000.0,1334687,1165312
